Now we want to populate our data repository

In [1]:
import json
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
import simplesimdb as simplesim # this one is our own database manager
import subprocess # to capture errors from the simulations
import itertools

In [2]:
m = simplesim.Manager(directory='advection', executable='./execute.sh', filetype='nc')

In [3]:
init2x= {"shear":[0.0,6.283185307179586],
         "mms": [-1.0,1.0]
         }
init2bc= {"shear":["PER", "PER"],
          "mms": ["DIR", "PER"]}

init2init =  {
    "shear": {"type": "shear", "rho": 0.20943951023931953, "delta": 0.2 },
    "mms":   {"type": "mms", "sigma": 0.2, "velocity": 1.0}
}

n2v={
     "shear": {
    1:[144,1e-2],
    3:[48, 5e-3],
    4:[36, 2.5e-3],
    5:[28, 2e-3],
    7:[20, 1e-3]
    },
    "mms": {
    1:[144,2e-2],
    3:[48, 2e-3],
    4:[36, 2e-3],
    5:[28, 1e-3],
    7:[20, 5e-4]
    }
}

endtime = {"shear": 10, "mms": 1}

maxout = {"shear": 40, "mms": 5}
timefactor = {"none" : 1, "modal" : 2,
              "viscous" : 1, "hyperviscous" : 1}
reg2reg = {"none" : {"type" : "none"},
           "modal" : {"type" : "modal", "alpha" : 36, "order" : 8, "eta_c" : 0.5},
          "viscous": {"type" : "viscosity", "order" : 1, "nu_perp" : 1e-3},
          "hyperviscous" : {"type": "viscosity", "order" : 2, "nu_perp" : 5e-4}
          }

In [4]:
def generate_standard_input( n, init, multiplication, advection, regularization) :
    itstp=endtime[init]/maxout[init]/n2v[init][n][1]
    if not itstp.is_integer() :
        print("Warning: itstp is not an integer!")
    inputfile={
        "grid":
        {   
            "type": "Cartesian2d",
            "n"  : n,
            "Nx" : n2v[init][n][0],
            "Ny" : n2v[init][n][0],
            "x": init2x[init],
            "y": init2x[init],
            "bc" : init2bc[init]
        },
        "timestepper":
        {   
            "stepper": "FilteredMultistep",
            "order": ["eBDF", 3],
            "dt" : n2v[init][n][1]
        },
        "regularization":  reg2reg[regularization],
        "output":
        {   
            "type": "netcdf",
            "itstp"   : int(itstp),
            "maxout"  : int(maxout[init])
        },
        "advection":
        {   
            "multiplication": multiplication,
            "type": advection
        },
        "elliptic":
        {   
            "type" : "multigrid",
            "stages": 3,
            "eps_pol" : [1e-6,10.0,10.0]
        },
        "init": init2init[init]
    }
    if regularization == "viscous" or regularization == "hyperviscous" :
        inputfile["timestepper"]["stepper"] = "Karniadakis"
        inputfile["timestepper"]["order"] = 3
        inputfile["timestepper"]["eps_time"] = 1e-9
    return inputfile

In [5]:
v_n=[1,3,4,5,7]
v_init=["shear", "mms"]
v_multiplication=["pointwise", "projection"]
v_advection=["arakawa","centered","upwind"]
v_regularization=["none", "modal", "viscous", "hyperviscous"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4])      

    ncfile=''
    # Check for existing data
    if not m.exists(inputfile):
        print(t)
    # uncomment the following to create data
    #try:
    #    print(t)
    #    ncfile=m.create(inputfile)
    # the simulation may fail 
    #except subprocess.CalledProcessError as e:
    #    print( e.stderr)
    #print( ncfile)

(1, 'mms', 'pointwise', 'upwind', 'none')
(1, 'mms', 'pointwise', 'upwind', 'modal')
(1, 'mms', 'pointwise', 'upwind', 'viscous')
(1, 'mms', 'projection', 'upwind', 'none')
(1, 'mms', 'projection', 'upwind', 'modal')
(1, 'mms', 'projection', 'upwind', 'viscous')
(3, 'shear', 'pointwise', 'upwind', 'viscous')
(3, 'shear', 'pointwise', 'upwind', 'hyperviscous')
(3, 'shear', 'projection', 'upwind', 'viscous')
(3, 'shear', 'projection', 'upwind', 'hyperviscous')
(5, 'shear', 'projection', 'upwind', 'viscous')
(7, 'shear', 'pointwise', 'arakawa', 'none')
(7, 'shear', 'pointwise', 'arakawa', 'viscous')
(7, 'shear', 'pointwise', 'upwind', 'viscous')
(7, 'shear', 'pointwise', 'upwind', 'hyperviscous')
(7, 'shear', 'projection', 'upwind', 'none')
(7, 'shear', 'projection', 'upwind', 'modal')
(7, 'shear', 'projection', 'upwind', 'viscous')
(7, 'shear', 'projection', 'upwind', 'hyperviscous')


In [7]:
# change the eta_c parameter to 0 for comparison
v_n=[3,4,5,7]
v_init=["shear"]
v_multiplication=["pointwise","projection"]
v_advection=["arakawa","centered"]
v_regularization=[ "modal"]

for t in itertools.product( v_n, v_init, v_multiplication, v_advection,
                           v_regularization) :
    inputfile = generate_standard_input( 
        n=t[0], init=t[1],multiplication=t[2],
        advection=t[3],regularization=t[4] ) 
    inputfile["regularization"]["eta_c"] = float(0)

    ncfile=''
    try:
        print(t)
        ncfile=m.create(inputfile)
    # the simulation may fail 
    except subprocess.CalledProcessError as e:
        print( e.stderr)

(3, 'shear', 'pointwise', 'arakawa', 'modal')
(3, 'shear', 'pointwise', 'centered', 'modal')
(3, 'shear', 'projection', 'arakawa', 'modal')
(3, 'shear', 'projection', 'centered', 'modal')
(4, 'shear', 'pointwise', 'arakawa', 'modal')
(4, 'shear', 'pointwise', 'centered', 'modal')
(4, 'shear', 'projection', 'arakawa', 'modal')
(4, 'shear', 'projection', 'centered', 'modal')
(5, 'shear', 'pointwise', 'arakawa', 'modal')
(5, 'shear', 'pointwise', 'centered', 'modal')
(5, 'shear', 'projection', 'arakawa', 'modal')
(5, 'shear', 'projection', 'centered', 'modal')
(7, 'shear', 'pointwise', 'arakawa', 'modal')
(7, 'shear', 'pointwise', 'centered', 'modal')
(7, 'shear', 'projection', 'arakawa', 'modal')
(7, 'shear', 'projection', 'centered', 'modal')
